Genetic algorithms work on the principle of Darwinian evolution. According to Darwinian evolution there are three key concepts that need to be in place for evolution to happen. The first is heredity. That is, there must be a procedure in place that allows "parents" to pass on information to thier "children". The second is variation. That is, we must have either a population of significant size, or some way to mutate individuals to introduce some variation. Finally we have selection. This is sometimes referred to as "survival of the fittest". This means that more adept, or superior, individuals are more likely to survive and pass down their genetic material. The genetic algorithm encodes these behaviours as `recombine`, `mutate` and `selection`.

In [2]:
from utils import *

def init_population(pop_number, gene_pool, state_length):
    """Initializes population for genetic algorithm
    pop_number  :  Number of individuals in population
    gene_pool   :  List of possible values for individuals
    state_length:  The length of each individual"""
    g = len(gene_pool)
    population = []
    for i in range(pop_number):
        new_individual = [gene_pool[random.randrange(0, g)] for j in range(state_length)]
        population.append(new_individual)

    return population

def fitness_threshold(fitness_fn, f_thres, population):
    if not f_thres:
        return None

    fittest_individual = max(population, key=fitness_fn)
    if fitness_fn(fittest_individual) >= f_thres:
        return fittest_individual

    return None

def genetic_algorithm(population, fitness_fn, gene_pool=[0, 1], f_thres=None, ngen=1000, pmut=0.1):
    """[Figure 4.8]"""
    for i in range(ngen):
        population = [mutate(recombine(*select(2, population, fitness_fn)), gene_pool, pmut)
                      for i in range(len(population))]

        fittest_individual = fitness_threshold(fitness_fn, f_thres, population)
        if fittest_individual:
            return fittest_individual


    return max(population, key=fitness_fn)

def select(r, population, fitness_fn):
    fitnesses = map(fitness_fn, population)
    sampler = weighted_sampler(population, fitnesses)
    return [sampler() for i in range(r)]

def recombine(x, y):
    n = len(x)
    c = random.randrange(0, n)
    return x[:c] + y[c:]

def mutate(x, gene_pool, pmut):
    if random.uniform(0, 1) >= pmut:
        return x

    n = len(x)
    g = len(gene_pool)
    c = random.randrange(0, n)
    r = random.randrange(0, g)

    new_gene = gene_pool[r]
    return x[:c] + [new_gene] + x[c + 1:]

Now we use the above code on some examples. We are going to start with a gene pool consisting of upper case characters, lower case characters and space character. Our aim is to obtain the below target string. The genetic algorithm process will select two of the best "individuals" (strings of characters), recombine them and mutate them. Each of these recombined and mutated individuals become members of the new population and the process continues for a specified number of iterations. Of course the algorithm has no idea what it means for an individual to be the best, so we have to provide a `fitness_fn`.

In [3]:
target = 'Anthony is cool'

# The ASCII values of uppercase characters ranges from 65 to 91
u_case = [chr(x) for x in range(65, 91)]
# The ASCII values of lowercase characters ranges from 97 to 123
l_case = [chr(x) for x in range(97, 123)]

gene_pool = []
gene_pool.extend(u_case) # adds the uppercase list to the gene pool
gene_pool.extend(l_case) # adds the lowercase list to the gene pool
gene_pool.append(' ')   

max_population = 100
mutation_rate = 0.07 # 7%

def fitness_fn(sample):
    # initialize fitness to 0
    fitness = 0
    for i in range(len(sample)):
        # increment fitness by 1 for every matching character
        if sample[i] == target[i]:
            fitness += 1
    return fitness


population = init_population(max_population, gene_pool, len(target))
result = genetic_algorithm(population, fitness_fn, gene_pool, f_thres=None, ngen=1000, pmut=0.1)
print(result)

['A', 'n', 't', 'h', 'o', 'n', 'y', ' ', 'i', 's', ' ', 'c', 'o', 'o', 'l']


My next example is a more complex version of that which appears in the AIMA repo.

In [14]:
edges = {
    'A': [0, 1],
    'B': [0, 3],
    'C': [1, 2],
    'D': [2, 3],
    'E': [0, 4],
    'F': [1, 4],
    'G': [2, 4],
    'H': [3, 4],
    'I': [5, 0],
    'J': [5, 1],
    'K': [5, 4]
}

gene_pool = ['R', 'G', 'B', 'Y']
population = init_population(50, gene_pool, 6)

def fitness(c):
    return sum(c[n1] != c[n2] for (n1, n2) in edges.values())

solution = genetic_algorithm(population, fitness, gene_pool)
print(solution)

['G', 'B', 'G', 'Y', 'R', 'Y']


By sketching a little diagram, we can see that this solution is correct. Now let's look at the N-queens problem.

In [22]:
population = init_population(1000, range(8), 8)

def fitness(q):
    non_attacking = 0
    for row1 in range(len(q)):
        for row2 in range(row1+1, len(q)):
            col1 = int(q[row1])
            col2 = int(q[row2])
            row_diff = row1 - row2
            col_diff = col1 - col2

            if col1 != col2 and row_diff != col_diff and row_diff != -col_diff:
                non_attacking += 1

    return non_attacking

solution = genetic_algorithm(population, fitness, f_thres=25, gene_pool=range(8), ngen=1000, pmut = 0.1)
print(solution)
print(fitness(solution))

[3, 0, 7, 1, 6, 7, 2, 4]
26


To get a better understanding of what this means, we need to do a little math. The fitness function calcualtes the number of pairs of non-attacking queens. If there were no attacking pairs of queens then the number of pairs would be the number of distinct pairs we can select from 8. This is 8C2, or (8 * 7) / 2 = 28. So 28 is the best possible score. Saying that, 26 is not that bad. We have set the function threshold to 25.